# Challenge Large Scale Machine Learning
## Fusion of algorithms for face recognition
Authors:
Pavlo Mozharovskyi (pavlo.mozharovskyi@telecom-paris.fr), Umut Şimşekli (umut.simsekli@telecom-paris.fr)



The increasingly ubiquitous presence of biometric solutions and face recognition in particular in everyday life requires their adaptation for practical scenario. In the presence of several possible solutions, and if global decisions are to be made, each such single solution can be far less efficient than tailoring them to the complexity of an image.

In this challenge, the goal is to build a fusion of algorithms in order to construct the best suited solution for comparison of a pair of images. This fusion will be driven by qualities computed on each image.

Comparing of two images is done in two steps. 1st, a vector of features is computed for each image. 2nd, a simple function produces a vector of scores for a pair of images. The goal is to create a function that will compare a pair of images based on the information mentioned above, and decide whether two images belong to the same person.

You are provided a label set of training data and a test set without labels. You should submit a .csv file with labels for each entry of this test set.

<h2 id="The-properties-of-the-dataset:">The properties of the dataset:<a class="anchor-link" href="#The-properties-of-the-dataset:">&#182;</a></h2>
</div>
</div>
</div>

</div>
<div class="text_cell_render border-box-sizing rendered_html">
<h3 id="Training-data:">Training data:<a class="anchor-link" href="#Training-data:">&#182;</a></h3>
</div>
</div>
</div>
</div>

<p>The training set consist of two files, <strong>xtrain_challenge.csv</strong> and <strong>xtest_challenge.csv</strong>.</p>
<p>File <strong>xtrain_challenge.csv</strong> contains one observation per row which contains following entries based on a pair of images:</p>
<ul>
<li>columns 1-13 - 13 qualities on first image;</li>
<li>columns 14-26 - 13 qualities on second image;</li>
<li>columns 27-37 - 11 matching scores between the two images.</li>
</ul>
<p>File <strong>ytrain_challenge.csv</strong> contains one line with each entry corresponding to one observation in <strong>xtrain_challenge.csv</strong>, maintaining the order, and has '1' if a pair of images belong to the same person and '0' otherwise.</p>
<p>For each of these 38 variables, there are in total 9,800,713 training observations.</p>

</div>
</div>

<div >
<h3 id="Test-data:">Test data:<a class="anchor-link" href="#Test-data:">&#182;</a></h3>
</div>

<p>File <strong>xtest_challenge.csv</strong> has the same structure as file <strong>xtrain_challenge.csv</strong>.</p>
<p>There are in total 3,768,311 test observations.</p>

<h2 id="The-performance-criterion&#182;">The performance criterion&#182;</h2>
Consider the problem of the supervised classification with two classes labeled '0' and '1'. Many methods for supervised classification assign a new observation $\boldsymbol{x}$ to a class using the following rule:</p>
\begin{align}
g(\boldsymbol{x}) = \begin{cases}1 &amp; \text{ if } f(\boldsymbol{x}) \ge t, \\ 0 &amp; \text{ otherwise}.\end{cases}
\end{align}<p>Threshold $t$ is then chosen due to specific needs managing the trade-off between the true positive rate (TPR) and the false positive rate (FPR), depending on the cost of the corresponding mistakes.</p>
<p>Here, the performance criterion is <strong>TPR for the value of FPR = $10^{-4}$</strong>, or, speaking in other words, one needs to maximize the value of the receiver operating characteristic (ROC) in the point FPR = $10^{-4}$. The submitted solution file should thus contain the score for each observation.

## imports 
### Pythons packages

In [ ]:
%matplotlib inline
### General imports ###
import pandas as pd
import numpy as np
from scipy import stats

### Visualization ###
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
#from ggplot import *
from matplotlib.collections import EllipseCollection
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

###metrics###
from sklearn.metrics import cohen_kappa_score

### Build the model ###
import pickle
from joblib import dump, load

### Machine Learning ###
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import KFold

### Mount google drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### import datasets
#### on Google Colab

In [ ]:
# Load training data
nrows_train = 9000000
nrows_valid = 800713
xtrain = np.loadtxt('/content/drive/My Drive/DataChallenge2020MSBGD/xtrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_valid)
ytrain = np.loadtxt('/content/drive/My Drive/DataChallenge2020MSBGD/ytrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_valid)
ytrain = np.array(ytrain).reshape(nrows_train + nrows_valid)

ytrain = np.array(ytrain).reshape(nrows_train + nrows_valid)

x_train=xtrain[:nrows_train]
y_train=ytrain[:nrows_train]
x_test=xtrain[nrows_train:(nrows_train + nrows_valid)] 
y_test=ytrain[nrows_train:(nrows_train + nrows_valid)] 

# Check the number of observations and properties

print(x_train.shape)
print(x_test.shape)

#### on local Notebook

In [ ]:
# Load training data
nrows_train = 8000
nrows_valid = 100000 #800713
xtrain = np.loadtxt('xtrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_valid)
ytrain = np.loadtxt('ytrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_valid)

ytrain = np.array(ytrain).reshape(nrows_train + nrows_valid)

x_train=xtrain[:nrows_train]
y_train=ytrain[:nrows_train]
x_test=xtrain[nrows_train:(nrows_train + nrows_valid)] 
y_test=ytrain[nrows_train:(nrows_train + nrows_valid)] 

# Check the number of observations and properties
print(xtrain[:1,0:12]) #1-13  #13
print(xtrain[:1,13:25]) #14-26  #13
print(xtrain[:1,26:36]) #27-37   #11
print(xtrain.shape)
print(ytrain.shape)

# Exploratory data analysis
## Duplicate and missing values detection

In [ ]:
x_train_df = pd.DataFrame(x_train[:800], columns=['fA1', 'fA2', 'fA3', 'fA4', 'fA5', 'fA6', 'fA7', 'fA8', 'fA9', 'fA10', 'fA11',
                                            'fA12', 'fA13', 'fB1', 'fB2', 'fB3', 'fB4', 'fB5', 'fB6', 'fB7', 'fB8', 'fB9', 'fB10', 'fB11', 'fB12', 'fB13',
                                            'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11'])
y_train_df = pd.DataFrame(y_train[:800])

x_test_df = pd.DataFrame(x_test[:800], columns=['fA1', 'fA2', 'fA3', 'fA4', 'fA5', 'fA6', 'fA7', 'fA8', 'fA9', 'fA10', 'fA11',
                                          'fA12', 'fA13', 'fB1', 'fB2', 'fB3', 'fB4', 'fB5', 'fB6', 'fB7', 'fB8', 'fB9', 'fB10', 'fB11', 'fB12', 'fB13',
                                          'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11'])

In [ ]:
a = np.array(x_train[:, 4])
np.unique(a, axis=0)
u, indices, counts = np.unique(a, return_index=True, return_counts=True)
print(len(indices))

In [ ]:
uc = x_train_df.nunique()
uc

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Parcoords(
    line=dict(color=x_train[:, 0],
              colorscale='Electric',
              showscale=True,
              ),
    dimensions=list([
        dict(range=[x_train[:, 0].min(), x_train[:, 0].max()],
             label="fA1", values=x_train[:, 0]),
        dict(range=[x_train[:, 1].min(), x_train[:, 1].max()],
             label='fA2', values=x_train[:, 1]),
        dict(range=[x_train[:, 2].min(), x_train[:, 2].max()],
             tickvals=[0, 0.5, 1, 2, 3],
             ticktext=['A', 'AB', 'B', 'Y', 'Z'],
             label='fA3', values=x_train[:, 2]),
        dict(range=[x_train[:, 3].min(), x_train[:, 3].max()],
             tickvals=[0, 1, 2, 3],
             label='fA4', values=x_train[:, 3]),
        dict(range=[x_train[:, 4].min(), x_train[:, 4].max()],
             visible=True,
             label='fA5', values=x_train[:, 4]),
        dict(range=[x_train[:, 5].min(), x_train[:, 5].max()],
             label='fA6', values=x_train[:, 5]),
        dict(range=[x_train[:, 6].min(), x_train[:, 6].max()],
             label='fA7', values=x_train[:, 6]),
        dict(range=[x_train[:, 7].min(), x_train[:, 7].max()],
             label='fA 8 ', values=x_train[:, 7]),
        dict(range=[x_train[:, 8].min(), x_train[:, 8].max()],
             label="fA 9 ", values=x_train[:, 8]),
        dict(range=[x_train[:, 9].min(), x_train[:, 9].max()],
             label="fA 10 ", values=x_train[:, 9]),
        dict(range=[x_train[:, 10].min(), x_train[:, 10].max()],
             label="fA 11 ", values=x_train[:, 10]),
        dict(range=[x_train[:, 11].min(), x_train[:, 11].max()],
             label="fA 12 ", values=x_train[:, 11]),
        dict(range=[x_train[:, 12].min(), x_train[:, 12].max()],
             label="fA 13 ", values=x_train[:, 12]),
    ]),
)
)
fig.show()

fig = go.Figure(data=go.Parcoords(
    line=dict(color=x_train[:, 13],  # colorscale='Electric',
              showscale=True,
              ),
    dimensions=list([
        dict(range=[x_train[:, 13].min(), x_train[:, 13].max()],
             label="fB 1 ", values=x_train[:, 13]),
        dict(range=[x_train[:, 14].min(), x_train[:, 14].max()],
             label="fB 2 ", values=x_train[:, 14]),
        dict(range=[x_train[:, 15].min(), x_train[:, 15].max()],
             label="fB 3 ", values=x_train[:, 15]),
        dict(range=[x_train[:, 16].min(), x_train[:, 16].max()],
             label="fB 4 ", values=x_train[:, 16]),
        dict(range=[x_train[:, 17].min(), x_train[:, 17].max()],
             label="fB 5 ", values=x_train[:, 17]),
        dict(range=[x_train[:, 18].min(), x_train[:, 18].max()],
             label="fB 6 ", values=x_train[:, 18]),
        dict(range=[x_train[:, 19].min(), x_train[:, 19].max()],
             label="fB 7 ", values=x_train[:, 19]),
        dict(range=[x_train[:, 20].min(), x_train[:, 20].max()],
             label="fB 8 ", values=x_train[:, 20]),
        dict(range=[x_train[:, 21].min(), x_train[:, 21].max()],
             label="fB 9 ", values=x_train[:, 21]),
        dict(range=[x_train[:, 22].min(), x_train[:, 22].max()],
             label="fB 10 ", values=x_train[:, 22]),
        dict(range=[x_train[:, 23].min(), x_train[:, 23].max()],
             label="fB 11 ", values=x_train[:, 23]),
        dict(range=[x_train[:, 24].min(), x_train[:, 24].max()],
             label="fB 12 ", values=x_train[:, 24]),
        dict(range=[x_train[:, 25].min(), x_train[:, 25].max()],
             label="fB 13 ", values=x_train[:, 25]),
    ]),
)
)
fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Parcoords(
    line=dict(  color = y_train,#colorscale='Electric',
        showscale=True,
             ) ,
    dimensions=list([
        dict(range=[(x_train[:, 0]-x_train[:, 13]).min(), (x_train[:, 0]-x_train[:,
                                                                                 13]).max()], label="fA-fB 0 ", values=x_train[:, 0]-x_train[:, 13]),
        dict(range=[(x_train[:, 1]-x_train[:, 14]).min(), (x_train[:, 1]-x_train[:,
                                                                                 14]).max()], label="fA-fB 1 ", values=x_train[:, 1]-x_train[:, 14]),
        dict(range=[(x_train[:, 2]-x_train[:, 15]).min(), (x_train[:, 2]-x_train[:,
                                                                                 15]).max()], label="fA-fB 2 ", values=x_train[:, 2]-x_train[:, 15]),
        dict(range=[(x_train[:, 3]-x_train[:, 16]).min(), (x_train[:, 3]-x_train[:,
                                                                                 16]).max()], label="fA-fB 3 ", values=x_train[:, 3]-x_train[:, 16]),
        dict(range=[(x_train[:, 4]-x_train[:, 17]).min(), (x_train[:, 4]-x_train[:,
                                                                                 17]).max()], label="fA-fB 4 ", values=x_train[:, 4]-x_train[:, 17]),
        dict(range=[(x_train[:, 5]-x_train[:, 18]).min(), (x_train[:, 5]-x_train[:,
                                                                                 18]).max()], label="fA-fB 5 ", values=x_train[:, 5]-x_train[:, 18]),
        dict(range=[(x_train[:, 6]-x_train[:, 19]).min(), (x_train[:, 6]-x_train[:,
                                                                                 19]).max()], label="fA-fB 6 ", values=x_train[:, 6]-x_train[:, 19]),
        dict(range=[(x_train[:, 7]-x_train[:, 20]).min(), (x_train[:, 7]-x_train[:,
                                                                                 20]).max()], label="fA-fB 7 ", values=x_train[:, 7]-x_train[:, 20]),
        dict(range=[(x_train[:, 8]-x_train[:, 21]).min(), (x_train[:, 8]-x_train[:,
                                                                                 21]).max()], label="fA-fB 8 ", values=x_train[:, 8]-x_train[:, 21]),
        dict(range=[(x_train[:, 9]-x_train[:, 22]).min(), (x_train[:, 9]-x_train[:,
                                                                                 22]).max()], label="fA-fB 9 ", values=x_train[:, 9]-x_train[:, 22]),
        dict(range=[(x_train[:, 10]-x_train[:, 23]).min(), (x_train[:, 10]-x_train[:,
                                                                                   23]).max()], label="fA-fB 10 ", values=x_train[:, 10]-x_train[:, 23]),
        dict(range=[(x_train[:, 11]-x_train[:, 24]).min(), (x_train[:, 11]-x_train[:,
                                                                                   24]).max()], label="fA-fB 11 ", values=x_train[:, 11]-x_train[:, 24]),
        dict(range=[(x_train[:, 12]-x_train[:, 25]).min(), (x_train[:, 12]-x_train[:,
                                                                                   25]).max()], label="fA-fB 12 ", values=x_train[:, 12]-x_train[:, 25]),
    
        dict(range=[-0.1,1], label="y", values=y_train),
    
    
    ]),
)
)

fig.show()

In [ ]:
sns.set()
plt.figure(figsize=(15, 5))
plt.bar(range(len(uc)), uc, label='number of unique values',)
#plt.bar(range(len(clf.feature_importances_)),clf.feature_importances_,label='model with new features',alpha=0.6)
plt.xticks(range(len(uc)), ('fA1', 'fA2', 'fA3', 'fA4', 'fA5', 'fA6', 'fA7', 'fA8', 'fA9', 'fA10', 'fA11',
                            'fA12', 'fA13', 'fB1', 'fB2', 'fB3', 'fB4', 'fB5', 'fB6', 'fB7', 'fB8', 'fB9', 'fB10', 'fB11', 'fB12', 'fB13',
                                            'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11'))
plt.legend()

In [ ]:
p1_unique = x_train_df.iloc[:, :12].drop_duplicates()
p2_unique = x_train_df.iloc[:, 13:25].drop_duplicates()
p1_n = len(p1_unique)
p2_n = len(p2_unique)
n = len(ytrain)

print("The n of unique picture 1: ", p1_n, "duplication : ", 1 - p1_n/n)
print("The n of unique picture 2: ", p2_n, "duplication : ", 1 - p2_n/n)

## Distribution of each variable/feature

In [ ]:
fig = plt.figure(figsize =(15,15))
x_df=pd.DataFrame(x_train[:100,])
sns.pairplot(x_df,  palette = "colorblind")
plt.show()

In [ ]:
# we plot to see the correlation between each feature and the labels
sns.set()
plt.figure(figsize=(16, 30))
j = 0
for i in range(0, 37, 1):
    j = j+1
    plt.subplot(13, 3, j)
    plt.plot(x_train[:, i], y_train)

In [ ]:
sns.set()
plt.figure(figsize=(16, 30))
j = 0
for i in range(0, 37, 1):
    j = j+1
    plt.subplot(13, 3, j)
    plt.scatter(xtrain[:, i], ytrain, alpha=0.05)
    plt.scatter(xtrain[:, i][ytrain == 1].mean(), 1, label='mean of class 1')
    plt.scatter(xtrain[:, i][ytrain == 0].mean(), 0, label='mean of class 0')
    plt.title('feature '+str(i))
    plt.legend()

In [ ]:
sns.set()
ytrain[xtrain[:, i] == 1]
plt.figure(figsize=(16, 22))
j = 0
for i in range(0, 14, 1):
    j = j+1
    plt.subplot(14, 2, j)
    plt.hist(xtrain[:, i], density=True)
    j = j+1
    plt.subplot(14, 2, j)
    plt.hist(xtrain[:, i+13], density=True)

In [ ]:
sns.set()
ytrain[xtrain[:, i] == 1]
plt.figure(figsize=(16, 10))
for i in range(11):
    plt.subplot(4, 3, i+1)
    plt.hist(xtrain[:, i+26], density=True)

## Visualisation with PCA

In [ ]:
from sklearn.decomposition import PCA
pca = decomposition.PCA(n_components=10)

In [ ]:
df_feat_pca = pca.fit_transform(x_train)
print('percent of the variance '+ str(sum(pca.explained_variance_ratio_)))

In [ ]:
pca = PCA(n_components=2).fit(x_train)
datapoint = pca.transform(x_train)
print('percent of the variance '+ str(sum(pca.explained_variance_ratio_)))

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(datapoint[:, 0], datapoint[:, 1], c=y_train, cmap='Set1')
plt.show()

In [ ]:
pca = PCA(n_components=3).fit(x_train)
datapoint = pca.transform(x_train)
print('percent of the variance '+ str(sum(pca.explained_variance_ratio_)))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D 

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(datapoint[:, 0], datapoint[:, 1], datapoint[:, 2],c  = y_train,cmap='Set1')
plt.show()

## Visualisation with TSNE

In [ ]:
from sklearn.manifold import TSNE
tSNE = TSNE(n_components=2)

#with 8000 data
datapoint = tSNE.fit_transform(x_train)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(datapoint[:, 0], datapoint[:, 1],c  = y_train,cmap='Set1')
plt.show()

In [ ]:
tSNE = TSNE(n_components=3)
datapoint = tSNE.fit_transform(x_train)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(datapoint[:, 0], datapoint[:, 1],
           datapoint[:, 2], c=y_train, cmap='Set1')

ax.view_init(elev=10., azim=210)
plt.show()

# Feature Engineering 

## Data augementation with Permutation of columns of image A and image B 
As we see, the col 0-12 and col 13-25 represent respectively pic1 and pic2, however, the machine learning algorithm doesn’t necessarily has this knowledge and would consider each feature a irrelevant feature at the beginning of learning. Even after learning it may not catch this pattern. In order to cope with this problem, I firstly do the swap of pic 1 and pic2 every two lines. By doing so I improved 97% to 98%  in terms of validation score. Soon I choose another better solutaion.

I make a hypothesis that the matching scores are symmetric similarities, which means the position of pic 1 and pic 2 shouldn't change the similarity measure. I have Score(pic 1 , pic 2) = Score( pic 2 , pic 1 ).

Then I duplicate x_train and swap pic 1 and pic 2, and added it back into x_train. Thus I doubled the dataset. I took 9M as training set, now I have 18M data. This not only  let the algorithmes be aware of the subsets pic1 and pic2, but also added one time more useful data. By doing so I get a score around 99.8% on validation set.


| col 0-12 | col 13-25   | matching scores  |label|
|----       --|------|------|------|
| PIC 1 | PIC 2   |  scores  |y|
|------|------|------|------|
|   PIC 2       | PIC 1| scores   |y|

The result is very surprising, showing that this method is very efficient and my hypothesis is true. 


In [ ]:
# duplicate x_train
def data_augementation(x_train,y_train):
    z = np.copy(x_train)

    # Permutation of pic 1 and pic 2
    x_train[:, [0, 13]] = x_train[:, [13, 0]]
    x_train[:, [1, 14]] = x_train[:, [14, 1]]
    x_train[:, [2, 15]] = x_train[:, [15, 2]]
    x_train[:, [3, 16]] = x_train[:, [16, 3]]
    x_train[:, [4, 17]] = x_train[:, [17, 4]]
    x_train[:, [5, 18]] = x_train[:, [18, 5]]
    x_train[:, [6, 19]] = x_train[:, [19, 6]]
    x_train[:, [7, 20]] = x_train[:, [20, 7]]
    x_train[:, [8, 21]] = x_train[:, [21, 8]]
    x_train[:, [9, 22]] = x_train[:, [22, 9]]
    x_train[:, [10, 23]] = x_train[:, [23, 10]]
    x_train[:, [11, 24]] = x_train[:, [24, 11]]
    x_train[:, [12, 25]] = x_train[:, [25, 12]]

    # stack x_train and permuted x_train
    x_train = np.vstack((x_train, z))
    y_train=np.hstack((y_train,y_train))
    return x_train,y_train

In [ ]:
y_train.shape

## Outlier Detection

- From the above distribution and correlation analysis, the column $(0,1,2,11,12,13,14,15,24,25)$ seems to have strange outliers when is far far from the $mean+-3*std$. Like the col 1, there are 0 values while mean = 0.99 and std is 0.06. 
I prefer to keep the outliers of other columns.

In [ ]:
xdf = pd.DataFrame(x_train, columns=['fA1', 'fA2', 'fA3', 'fA4', 'fA5', 'fA6', 'fA7', 'fA8', 'fA9', 'fA10', 'fA11',
                                    'fA12', 'fA13', 'fB1', 'fB2', 'fB3', 'fB4', 'fB5', 'fB6', 'fB7', 'fB8', 'fB9', 'fB10', 'fB11', 'fB12', 'fB13',
                                    'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11'])
xdf.describe()

In [ ]:
# detect the data with at least one outlier
# as default we only clean the columns 0, 1, 2, 11, 12, 13, 14, 15, 24, 25

def detect_outliers(xtrain, factor=3, r=None):
    outlier_indice = []
    if r == None:
        #r = range(26)
        r = (0, 1, 2, 11, 12, 13, 14, 15, 24, 25)
    else:
        r = r

    for i in r:
        upper_lim = xtrain[:, i].mean()+xtrain[:, i].std()*factor
        lower_lim = xtrain[:, i].mean()-xtrain[:, i].std()*factor

        # Determine a list of indices of outliers for feature col
        outlier_list_i = np.where(
            (xtrain[:, i] < lower_lim) | (xtrain[:, i] > upper_lim))

        first_list = outlier_indice
        second_list = list(outlier_list_i)
        # print(outlier_list_i)
        # print(len(outlier_list_i[0]))
        in_first = set(first_list)
        in_second = set(outlier_list_i[0])
        in_second_but_not_in_first = in_second - in_first

        outlier_indice = first_list + list(in_second_but_not_in_first)

    return outlier_indice

In [ ]:
# drop all the data with at least one outlier
def drop_outliers(xtrain, ytrain, factor=3, r=None):
    outlier_list = detect_outliers(xtrain, factor, r)
    xtrain_without_outliers = np.delete(xtrain, outlier_list, axis=0)
    ytrain_without_outliers = np.delete(ytrain, outlier_list, axis=0)
    print("used to have train data number", xtrain.shape[0])
    print("number of outliers", len(outlier_list))
    print("number of cleaned train data", ytrain_without_outliers.shape[0])
    return xtrain_without_outliers, ytrain_without_outliers

## Feature Reengineering
I add new features like subtraction and division between 2 pictures, this may help basic models to learn better the insights from the datas. I tested on Logistic Regression, the TPR before and after adding these features are 85% and 90% ( 200K/100K : train/validation ). 

However, for model with large capacity, adding these new features seems unnecessary. 

In [ ]:
# substraction and divistion of comparable features
substraction = x_train[:, 0:12]-x_train[:, 13:25]
divistion = x_train[:, 13:25]/(x_train[:, 0:12]+0.00000000000001)
x_train_new = np.c_[x_train, substraction, divistion]

substraction = x_test[:, 0:12]-x_test[:, 13:25]
divistion = x_test[:, 13:25]/(x_test[:, 0:12]+0.00000000000001)
x_test_new= np.c_[x_test, substraction, divistion]

print(x_train_new.shape)
print(x_test_new.shape)

In [ ]:
# test on WRF
clf0 = RandomForestClassifier(class_weight={0: 5800, 1: 1}, max_features='sqrt',
                             n_jobs=-1, bootstrap=True,  n_estimators=100)  # , random_state=2)
clf0.fit(x_train, y_train)
# validation
pred = clf0.predict(x_test)
pred_prob = clf0.predict_proba(x_test)[:, clf.classes_ == 1][:, 0]

true_positive_rate(pred_prob)

In [ ]:
# test on WRF
clf = RandomForestClassifier(class_weight={0: 5800, 1: 1}, max_features='sqrt',
                             n_jobs=-1, bootstrap=True,  n_estimators=100)  # , random_state=2)
clf.fit(x_train_new, y_train)
# validation
pred = clf.predict(x_test_new)
pred_prob = clf.predict_proba(x_test_new)[:, clf.classes_ == 1][:, 0]

true_positive_rate(pred_prob)

In [ ]:
clf.feature_importances_
sns.set()
plt.figure(figsize=(15,4))
plt.bar(range(len(clf.feature_importances_)),clf.feature_importances_,label='model with new features',alpha=0.6)
plt.plot(range(len(clf.feature_importances_)),clf.feature_importances_)
plt.bar(range(len(clf0.feature_importances_)),clf0.feature_importances_,label='model without new features',alpha=0.6)
plt.plot(range(len(clf0.feature_importances_)),clf0.feature_importances_)
plt.legend()

# Cost Sensitive Learning and our objective
## Analysis the objective
Let’s consider the confusion matrix for the two-class problem. The confusion matrix is given for example. In the two-class problem, if you correctly classify something as positive, it’s called a True Positive, and it’s called a True Negative when you properly classify the negative class.  The other two possible cases
 are False Negative and False Positive. 
 
| TN          | FN   |
|----       --|------|
|   FP       | TP(FP=<0.0001)|
 
- Our objective is to keep False Positive in the the zone in down left corner very low$FPR = 10^{−4}$, and maximize the TP in the down right corner. 
- However, there is a trade off between FP and TP. If we only minimise FP, the model will predict more negative, thus the True positive will decrease.  

In [ ]:
sns.heatmap(confusion_matrix(pred,y_test),annot=True,fmt='3.0f',cmap="coolwarm",linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

### Asymetric Cost of TP and FP

In many cases, we cann't assume that the cost of classifying things is
equal. For example, we build a system to detect whether a horse with stomach pain would end up living or dying.  Let’s say someone brings a horse to us and
asks us to predict whether the horse will live or die. We say die, and rather than delay
the inevitable, making the animal suffer and incurring veterinary bills, they have it
euthanized. Perhaps our prediction was wrong, and the horse would have lived. Our
classifier is only 80% accurate, after all. If we predicted this incorrectly, then an expensive animal would have been destroyed, not to mention that a human was emotionally
attached to the animal. 


In our case, let's image it's a face recognition for a bank, if we have false negative, then a client could be denied access, but if we have false positive, then we may let in some potential malefactors. To avoid the danger of let pass the wrong user, we only allow a false positive rate of $10^{−4}$. 


With these definitions we can define some new metrics that are more useful than
error rate when detection of one class is more important than another class. For example, we can use $Precision = TP/(TP+FP)$ and $Recall = TP/(TP+FN)$. However, why not use directly the given criterion as objective and evaluation function?

In our case, the performance criterion is TPR for the value of $FPR = 10^{−4}$, that's to say we want to maximize the $Recall = TP/(TP+FN)$ while keeping Precision higher than a threshold  $Precison = TP/(TP+FP) \approx 99.99\%$. 


### TPR

In [ ]:
# Calculate the performance

def true_positive_rate(yvalid):
    yvalid_scoreordered = y_test[np.argsort(yvalid)]
    #print(yvalid_scoreordered)
    N = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 0)
    P = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 1)
    FP = 0
    TP = 0
    for i in range(nrows_valid - 1, -1, -1):
        if (yvalid_scoreordered[i] == 1):
            TP = TP + 1
        else:
            FP = FP + 1
        if (FP / N > 10**-4):
            FP = FP - 1
            break
    print("For the smallest FPR <= 10^-4 (i.e., ",
          FP / N, ") TPR = ", TP / P, ".", sep="")

    #print("cohen_kappa_score :  ", cohen_kappa_score(y_test, yvalid), sep="")

    return TP/P

### ROC from scratch

Another tool used for measuring classification imbalance is the ROC curve. ROC
stands for receiver operating characteristic, and it was first used by electrical engineers building radar systems during World War II.  The ROC curve shows how the two rates change as the threshold changes. 

In order to plot the ROC I need the classifier to give a numeric score of how
positive or negative each instance is. Most classifiers give this to you. 
The input to the sigmoid in logistic regression is a numeric value. AdaBoost and SVMs
both compute a numeric value that’s input to the predic_proba() function. All of these values
can be used to rank how strong the prediction of a given classifier is. To build the ROC
curve, I first sort the instances by their prediction strength. I start with the lowest
ranked instance and predict everything below this to be in the negative class and everything above this to be the positive class. This corresponds to the point 1.0,1.0. I move
to the next item in the list, and if that is the positive class, I move the true positive rate,
but if that instance is in the negative class, I change the true negative rate:

In [ ]:
def ROC(yvalid):
    yvalid_scoreordered = y_test[np.argsort(yvalid)]
    #print(yvalid_scoreordered)
    N = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 0)
    P = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 1)
    FP = 0
    TP = 0
    FP_n=[]
    TP_n=[]
    for i in range(nrows_valid - 1, -1, -1):
        if (yvalid_scoreordered[i] == 1):
            TP = TP + 1
        else:
            FP = FP + 1
        FP_n.append(FP)
        TP_n.append(TP)
    sns.set()   
    plt.scatter(TP_n,FP_n,)
    plt.axvline(x= N * 10**-4)
    plt.xlabel('False positive')
    plt.ylabel('True positive')
    


In [ ]:
clf = LogisticRegression(solver='lbfgs')
clf.fit(x_train, y_train)
yvalid=clf.predict_proba(x_test)[:, 0]
ROC(yvalid)    

## Cost Sensitive Learning

### Manipulating the classifier’s decision with a cost function 
#### Weighted Random Forest

Besides tuning the thresholds of our classifier, there are other approaches
e to aid with uneven classification costs. One such method is known as costsensitive learning. 

For random forest I use the parameter of class_weight dict. If not given, all classes are supposed to have weight one.
Weights associated with classes in the form {class_label: weight}. The best I find with my hyperparameter tuning for Weighted Random Forest is around class_weight={0: 5000, 1: 1} (best TPR = 0.9464351139444567) (Before data augementation)
                                 
#### Boosting 

For boosting methodes, we can define custom objective function and custom evaluation. I will develop in section 5.1 and 5.2.


## Experimental Models
### Baseline - LogisticRegression - with all features

The first 26 features are mades directly from the 2 pictures, however, these features has few correlation that we can exploite directly from. I tested some features engineering on logistic regression model without any strong conviction.

In [ ]:
# 1 Train the classifier with all features
clf = LogisticRegression(solver='lbfgs')
clf.fit(x_train, y_train)

pred = clf.predict(x_test)
pred_prob = clf.predict_proba(x_test)[:, clf.classes_ == 1][:, 0]
print("Train the classifier with all features")
true_positive_rate(pred_prob)

# 2 Train the classifier with only features of 1-26 cols
clf = LogisticRegression(solver='lbfgs')
clf.fit(x_train[:, 1:25], y_train)

pred = clf.predict(x_test[:, 1:25])
pred_prob = clf.predict_proba(x_test[:, 1:25])[:, clf.classes_ == 1][:, 0]
print("Train the classifier with only features of 1-26 cols")
true_positive_rate(pred_prob)

# 3 Train the classifier features of 27-36 cols
clf = LogisticRegression(solver='lbfgs')
clf.fit(x_train[:, 26:], y_train)

pred = clf.predict(x_test[:, 26:])
pred_prob = clf.predict_proba(x_test[:, 26:])[:, clf.classes_ == 1][:, 0]
print("Train the classifier with features of 27-36 cols")
true_positive_rate(pred_prob)

In [ ]:
# 4 Train the classifier without outliers of 1-26 cols- with all features
xtrain_without_outliers, ytrain__without_outliers = drop_outliers(
    x_train, y_train, factor=3, r=(0, 1, 2, 11, 12, 13, 14, 15, 24, 25))

clf = LogisticRegression(solver='lbfgs')
clf.fit(xtrain_without_outliers, ytrain__without_outliers)

pred = clf.predict(x_test)
pred_prob = clf.predict_proba(x_test)[:, clf.classes_ == 1][:, 0]
print("Train the classifier without outliers of 1-26 cols- with all features")
true_positive_rate(pred_prob)

In [ ]:
sns.heatmap(confusion_matrix(pred, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

In [ ]:
# 4 Train the classifier without outliers of 1-26 cols- with all features
xtrain_without_outliers, ytrain__without_outliers = drop_outliers(
    x_train, y_train, factor=3, r=(0, 1, 2, 11, 12, 13, 14, 15, 24, 25))

clf = LogisticRegression(solver='lbfgs',class_weight={0: 5000, 1: 1})
clf.fit(xtrain_without_outliers, ytrain__without_outliers)

pred = clf.predict(x_test)
pred_prob = clf.predict_proba(x_test)[:, clf.classes_ == 1][:, 0]
print("Train the classifier without outliers of 1-26 cols- with all features")
true_positive_rate(pred_prob)

### LogisticRegression  - drop outliers

In [ ]:
xtrain_without_outliers,ytrain__without_outliers=drop_outliers(x_train,y_train ,factor=6)

In [ ]:
# Train the classifier on a part of the data set
clf = LogisticRegression(solver='lbfgs')
clf.fit(xtrain_without_outliers,ytrain__without_outliers)

pred = clf.predict(x_test)
pred_prob = clf.predict_proba(x_test)[:, clf.classes_ == 1][:, 0]

true_positive_rate(pred_prob)

### LogisticRegression - with 61 features

In [ ]:
clf = LogisticRegression(solver='saga',class_weight={0: 10000, 1: 1},n_jobs=-1)
clf.fit(x_train_new[:,27:],y_train)

pred = clf.predict(x_test_new[:,27:])
pred_prob = clf.predict_proba(x_test_new[:,27:])[:, clf.classes_ == 1][:, 0]

true_positive_rate(pred_prob)

# WRF Weighted Random Forest
Another approach to make random forest more suitable for learning from extremely imbalanced data follows the idea of cost sensitive learning. Since the RF classifier tends to be biased towards the majority class, we
shall place a heavier penalty on misclassifying the minority class. We assign a weight to each class, with the
minority class given larger weight (i.e., higher misclassification cost).


The class weights are incorporated
into the RF algorithm in two places. In the tree induction procedure, class weights are used to weight
the Gini criterion for finding splits. In the terminal nodes of each tree, class weights are again taken into
consideration. The class prediction of each terminal node is determined by “weighted majority vote”; i.e.,
the weighted vote of a class is the weight for that class times the number of cases for that class at the
terminal node. The final class prediction for RF is then determined by aggregatting the weighted vote from
each individual tree, where the weights are average weights in the terminal nodes. Class weights are an
essential tuning parameter to achieve desired performance. 

The out-of-bag estimate of the accuracy from
RF can be used to select weights. This method, Weighted Random Forest (WRF), is incorporated in sklearn.

In [ ]:
# Train the classifier on a part of the data set
clf = RandomForestClassifier(class_weight={
                             0: 6000, 1: 1}, max_features='sqrt',
                             n_jobs=-1, bootstrap=True,  n_estimators=100)  # , random_state=2)

clf.fit(x_train_new, y_train)

# Check its on another part of the data set
pred = clf.predict(x_test_new)
pred_prob = clf.predict_proba(x_test_new)[:, clf.classes_ == 1][:, 0]

true_positive_rate(pred_prob)

In [ ]:
#pred = clf.predict(x_test)
sns.heatmap(confusion_matrix(pred, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

In [ ]:
clf.feature_importances_
plt.plot(clf.feature_importances_)

In [ ]:
clf.feature_importances_
plt.plot(clf.feature_importances_)

In [ ]:
from joblib import dump, load
dump(clf, 'WRF6000_1_new.joblib')  #name rule : WRF1000_1

In [ ]:
clf = load('WRF20000_1.joblib') 

## Search for the best class_weight

In [ ]:
TPR = []
Feature_importance = []
r = range(1000, 8000, 200)
for i in r:

    clf = RandomForestClassifier(class_weight={
                                 0: i, 1: 1}, max_features='sqrt', n_jobs=-1, bootstrap=True, random_state=2, n_estimators=100)

    clf.fit(x_train, y_train)

    # Check its on another part of the data set
    yvalid = clf.predict_proba(x_test)[:, clf.classes_ == 1][:, 0]

    yvalid_scoreordered = y_test[np.argsort(yvalid)]

    N = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 0)
    P = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 1)
    FP = 0
    TP = 0
    for i in range(nrows_valid - 1, -1, -1):
        if (yvalid_scoreordered[i] == 1):
            TP = TP + 1
        else:
            FP = FP + 1
        if (FP / N > 10**-4):
            FP = FP - 1
            break
    TPR.append(TP/P)
    Feature_importance.append(clf.feature_importances_)

print("For the smallest FPR <= 10^-4, the best TPR = ", np.max(TPR), ".", sep="")
sns.set()
plt.figure(figsize=(15, 5))
plt.plot(r, TPR)

In [ ]:
sns.set()
plt.figure(figsize=(15,5))
plt.plot(TPR)

## Adaboost

In [ ]:
# Train the classifier on a part of the data set
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
base_estimator = DecisionTreeClassifier(
    max_depth=1, class_weight={0: 1000, 1: 1})  # class_weight={0: 800, 1: 1}
clf = AdaBoostClassifier(
    base_estimator=None, n_estimators=100, learning_rate=0.8, algorithm='SAMME.R', random_state=None)

clf.fit(x_train, y_train)

# Check its on another part of the data set
pred = clf.predict(x_test)
pred_prob = clf.predict_proba(x_test)[:, clf.classes_ == 1][:, 0]

true_positive_rate(pred_prob)

In [ ]:
true_positive_rate(yvalid)

In [ ]:
sns.heatmap(confusion_matrix(pred, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

## XGboost

XGboost use a second ordre Taylor approximation, so it requests the gradient and the hessien in its cost function. For binary classification, we use a log loss:
$$𝐿=−𝑦ln𝑝−𝛽(1−𝑦)ln(1−𝑝)$$ $p$ as the probability estimated by sigmoid function. $𝛽$ is the multiplier factor to increase the weight of FP loss.

In order to penalise False Positive, we put a penalty Beta on the FP, we can calculate the gradent as: 

$$grad =\frac{∂𝐿}{∂𝑥}=\frac{∂𝐿}{∂𝑝}\frac{∂𝑝}{∂𝑥}=𝑝(𝛽+𝑦−𝛽𝑦)−𝑦 $$,

and hessien as:

$$hess =∂2𝐿∂𝑥2=𝑝(1−𝑝)(𝛽+𝑦−𝛽𝑦)$$

In [ ]:
import xgboost as xgb
D_train = xgb.DMatrix(x_train, label=y_train)
D_valid = xgb.DMatrix(x_test, label=y_test)

In [ ]:
def weighted_logloss(dtrain, y_hat):
    y = dtrain
    p = y_hat
    beta = 500
    grad = p * (beta + y - beta*y) - y
    hess = p * (1 - p) * (beta + y - beta*y)
    return grad, hess

In [ ]:
def custom_loss_function(y, pred):
    beta = 0.01
    p = 1. / (1 + np.exp(-pred))
    grad = p * ((beta - 1) * y + 1) - beta * y
    hess = ((beta - 1) * y + 1) * p * (1.0 - p)

    return grad, hess

In [ ]:
eval_set = [(x_test, y_test)]

param = {'max_depth': 28, 'scale_pos_weight': 0.01, 'eta': 0.1,
         'objective': 'binary:logistic', 'eval_metric': 'error@0.8'}

gbm = xgb.XGBClassifier(max_depth=20, n_estimators=40, learning_rate=0.05,
                        objective=custom_loss_function,
                        eval_metric="logloss", eval_set=eval_set, verbose_eval=True,
                        early_stopping_rounds=10,
                        scale_pos_weight=0.07, random_state=2, verbosity=2, n_jobs=-1,).fit(x_train, y_train)
preds_proba = gbm.predict(x_test)

true_positive_rate(preds_proba)

sns.heatmap(confusion_matrix(preds_proba, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

In [ ]:
preds_proba = gbm.predict(x_test)

true_positive_rate(preds_proba)

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(y_test, preds_proba)

In [ ]:
sns.heatmap(confusion_matrix(preds__proba, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

In [ ]:
ax = xgb.plot_importance(bst)
fig = ax.figure
fig.set_size_inches(10, 5)

In [ ]:
ax= xgb.plot_tree(bst, num_trees=1)
fig = ax.figure
fig.set_size_inches(18, 10)

In [ ]:
# hyperparameter tuning
from sklearn.model_selection import GridSearchCV

clf = xgb.XGBClassifier()
parameters = {
    "eta": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_weight": [1, 3, 5, 7],
    "gamma": [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5, 0.7]
}

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, Y_train)

# (customized) Lightgbm 

## Custom Loss Function, penalty for FP

Xgboost use a second ordre Taylor approximation, very alike to Xgboost, light gbm also request the gradient and the hessien in its cost function. For binary classification, we use a log loss:
$$𝐿=−𝑦ln𝑝−𝛽(1−𝑦)ln(1−𝑝)$$ $p$ as the probability estimated by sigmoid function. $𝛽$ is the multiplier factor to increase the weight of FP loss.

In order to penalise False Positive, we put a penalty Beta on the FP, we can calculate the gradient as: 

$$grad =\frac{∂𝐿}{∂𝑥}=\frac{∂𝐿}{∂𝑝}\frac{∂𝑝}{∂𝑥}=𝑝(𝛽+𝑦−𝛽𝑦)−𝑦 $$,

and hessien as:

$$hess =∂2𝐿∂𝑥2=𝑝(1−𝑝)(𝛽+𝑦−𝛽𝑦)$$

 Customizing the training loss in LightGBM requires defining a function that takes in two arrays, the targets and their predictions. In turn, the function should return two arrays of the gradient and hessian of each observation. As noted above, we need to use calculus to derive gradient and hessian and then implement it in Python.

In [ ]:
def custom_loss_function(preds, train_data):
    # pred = raw value before logistic transformation <- 0 ->
    y = train_data.get_label()
    pred = preds
    beta = 5

    #p = 1. / (1. + np.exp(-pred)) to avoid exp overflow:
    p = np.where(preds >= 0, 1.0 / (1+np.exp(-preds)),
                 np.exp(preds)/(1+np.exp(preds)))
    grad = (p * (beta + y - beta * y) - y)
    hess = p*(1.0 - p) * (beta + y - beta * y)

    return grad, hess

In [ ]:
# test the cost function
y = np.asarray([1, 1, 0, 0])
pred = np.asarray([1, 0, 1, 0])
beta = 0.1

p = 1. / (1 + np.exp(-pred))
print("residu", pred-y)
print("p", p)

grad = (p * (beta + y - beta * y) - y)

hess = p*(1.0 - p) * (beta + y - beta * y)
print("grad", grad)
print("hess", hess)

## Custom Eval Metric TPR

Validation loss: This is the function that we use to evaluate the performance of our trained model on unseen data. This is often not the same as the training loss. For example, in the case of a classifier, this is often the area under the curve of the receiver operating characteristic (ROC) — though this is never directly optimized, because it is not differentiable. This is often called the “performance or evaluation metric”. The validation loss is often used to tune hyper-parameters. 

Because it doesn’t have as many functional requirements like the training loss does, the validation loss can be non-convex, non-differentiable, and discontinuous, we can use our Specific True Positive Rate (conditionned by FP < 10e-4) as validation loss. The validation loss in LightGBM is called metric.



In [ ]:
def TPR_eval(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    yvalid_scoreordered = labels[np.argsort(preds)]

    N = np.sum(y_test == 0)
    P = np.sum(y_test == 1)
    FP = 0
    TP = 0
    FN = 0
    TN = 0

    for i in range(len(labels) - 1, -1, -1):
        if (yvalid_scoreordered[i] == 1):
            TP = TP + 1
        else:
            FP = FP + 1
            #print("True Positive", TP, "False Positive", FP)
        if (FP / N > 10**-4):
            FP = FP - 1
            break

    # is_higher_better=True
    return "###TPR(FPR<0.0001)###:", TP/P, True

In [ ]:
# a very with more verbose for observation and debug
def TPR_eval_verbose(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    yvalid_scoreordered = y_test[np.argsort(preds)]
    preds_scoreordered = preds[np.argsort(preds)]

    N = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 0)
    P = np.sum(ytrain[nrows_train:(nrows_train + nrows_valid)] == 1)
    FP = 0
    TP = 0
    FN = 0
    TN = 0
    TP_total = 0
    FN_total = 0
    TN_total = 0
    FP_total = 0
    for i in range(len(labels) - 1, -1, -1):
        if (yvalid_scoreordered[i] == 1):
            TP = TP + 1
        else:
            FP = FP + 1
            print("True Positive", TP, "False Positive", FP)
        if (FP / N > 10**-4):
            FP = FP - 1
            break
    print("True Positive", TP, "False Positive", FP)
    # added
    for i in range(nrows_valid - 1, -1, -1):
        # print(preds_scoreordered[i])
        if (yvalid_scoreordered[i] == 0):
            if preds_scoreordered[i] < 0.5:
                TN_total = TN_total+1

            else:
                FP_total = FP_total+1
        if (yvalid_scoreordered[i] == 1):
            if preds_scoreordered[i] > 0.5:
                TP_total = TP_total+1
            else:
                FN_total = FN_total+1

    #print("cohen_kappa_score :  ", cohen_kappa_score(y_test, yvalid), sep="")
    print("____________________________")
    print("FP", FP_total, "FN:", FN_total, "FN/FP:", FN_total/FP_total)
    return "###TPR(FPR<0.0001)###:", TP/P,  True

In [ ]:
# Other eval metric
def accuracy(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    return 'accuracy', np.mean(labels == (preds > 0.5)), True

In [ ]:
#run with cumstom loss and eval
lgb_train = lgb.Dataset(data=x_train, label=y_train)
lgb_eval = lgb.Dataset(data=x_test, label=y_test, reference=lgb_train)

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    # 'objective' = custom_loss_function,
    # 'metric':{'12','auc','binary_logloss'},
    'num_leaves': 600,
    'num_trees': 300,
    'learning_rate': 0.05,
    # 'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'scale_pos_weight': 0.0969,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                num_boost_round=500,
                fobj=custom_loss_function,
                # feval=binary_error,
                feval=TPR_eval,
                #learning_rates=lambda iter: 0.05 * (0.99 ** iter),
                #feval=lambda preds, train_data: [accuracy(preds, train_data),TPR_eval(preds, train_data)],
                early_stopping_rounds=50)

print(gbm.best_score)
print('Starting predicting...')
# predict
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
print(y_pred)


In [ ]:
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)

gbm.best_score

print(y_pred)

true_positive_rate(y_pred)

### Prediction Analysis

In [ ]:
pred_l = 1. / (1. + np.exp(-y_pred))
y_scoreordered = y_test[np.argsort(pred_l)]
p_scoreordered = y_pred[np.argsort(pred_l)]
p_thershlod = p_scoreordered[-13000]
plt.figure(figsize=(12, 5))

FN = []
FP = []

for i in range(len(y_pred)):
    if y_pred[i] <= 0 and y_test[i] == 1:
        FN.append(y_pred[i])

    if y_pred[i] > 1 and y_test[i] == 0:
        FP.append(y_pred[i])

plt.figure(figsize=(12, 5))

sns.distplot(y_pred, hist=True, kde=False,
             bins=40, color='darkblue',
             hist_kws={'edgecolor': 'black'},
             kde_kws={'linewidth': 4})
plt.axvline(0)
plt.axvline(x=p_thershlod)
plt.scatter(FN, np.ones(len(FN)))
plt.scatter(FP, np.ones(len(FP)))
plt.show()

In [ ]:
preds_heatmap=np.where(y_pred>=0,1,0)
sns.heatmap(confusion_matrix(preds_heatmap, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

In [ ]:
print('Plotting metrics recorded during training...')
#ax = lgb.plot_metric(evals_result, metric='l1')
#plt.show()

print('Plotting feature importances...')
ax = lgb.plot_importance(gbm, max_num_features=40)
fig = ax.figure
fig.set_size_inches(10, 7)
plt.show()

print('Plotting split value histogram...')
ax = lgb.plot_split_value_histogram(gbm, feature='Column_29', bins='auto')
plt.show()

print('Plotting 54th tree...')  # one tree use categorical feature to split
ax = lgb.plot_tree(gbm, tree_index=53, figsize=(15, 15), show_info=['split_gain'])
plt.show()

print('Plotting 54th tree with graphviz...')
graph = lgb.create_tree_digraph(gbm, tree_index=53, name='Tree54')
#graph.render(view=True)

### drop_outliers

In [ ]:
xtrain_without_outliers, ytrain_without_outliers = drop_outliers(xtrain, ytrain, factor=3,r=(0,1,2,11,12,13,14,15,24,25))

def custom_loss_function(preds, train_data):
    # pred = raw value before logistic transformation <- 0 ->
    y = train_data.get_label()
    pred = preds
    beta = 5
    p = 1. / (1. + np.exp(-pred))
    grad = (p * (beta + y - beta * y) - y)
    hess = p*(1.0 - p) * (beta + y - beta * y)
    return grad, hess


lgb_train = lgb.Dataset(data=xtrain_without_outliers, label=ytrain_without_outliers)
lgb_eval = lgb.Dataset(data=x_test, label=y_test, reference=lgb_train)

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    #'objective': 'binary',
    #'metric':{'12','auc','binary_logloss','TPR_eval'},
    'num_leaves': 600,
    'learning_rate': 0.05,
    # 'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'scale_pos_weight': 0.0969,
    'bagging_freq': 5,
    'verbose': 0
}

evals_result = {}
print('Starting training...')
# train
gbm = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                num_boost_round=50,
                #fobj=custom_loss_function,
                # feval=binary_error,
                #feval=TPR_eval,
                feval=TPR_eval_verbose,
                evals_result=evals_result,
                #learning_rates=lambda iter: 0.05 * (0.99 ** iter),
                #feval=lambda preds, train_data: [accuracy(preds, train_data),TPR_eval(preds, train_data)],
                early_stopping_rounds=50)

print(gbm.best_score)
print('Starting predicting...')

# predict
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
print(y_pred)

# true_positive_rate(y_pred)

In [ ]:
preds_heatmap=np.where(y_pred>=0.5,1,0)
sns.heatmap(confusion_matrix(preds_heatmap, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

In [ ]:
pred_l = 1. / (1. + np.exp(-y_pred))
y_scoreordered = y_test[np.argsort(pred_l)]
p_scoreordered = y_pred[np.argsort(pred_l)]
p_thershlod=p_scoreordered[-13000]

FN=[]
FP=[]

for i in range(len(y_pred)):
        if y_pred[i] <= 0 and y_test[i] == 1:
            FN.append(y_pred[i])
            
        if y_pred[i] > 1 and y_test[i] == 0:
            FP.append(y_pred[i])

plt.figure(figsize=(12,5))

sns.distplot(y_pred, hist=True, kde=False, 
             bins=40, color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
plt.axvline(0)
plt.scatter(FN,np.ones(len(FN)))
plt.scatter(FP,np.ones(len(FP)))
plt.show()

In [ ]:
print('Plotting metrics recorded during training...')
ax = lgb.plot_metric( evals_result, metric='auc')
plt.show()

print('Plotting feature importances...')
ax = lgb.plot_importance(gbm, max_num_features=40)
fig = ax.figure
fig.set_size_inches(10, 7)
plt.show()

print('Plotting split value histogram...')
ax = lgb.plot_split_value_histogram(gbm, feature='Column_29', bins='auto')
plt.show()

print('Plotting 54th tree...')  # one tree use categorical feature to split
ax = lgb.plot_tree(gbm, tree_index=53, figsize=(15, 15), show_info=['split_gain'])
plt.show()

print('Plotting 54th tree with graphviz...')
graph = lgb.create_tree_digraph(gbm, tree_index=53, name='Tree54')
#graph.render(view=True)

In [ ]:
# save model to file
gbm.save_model('model_1.txt')

print('Dumping model to JSON...')
# dump model to JSON (and save to file)
model_1_json = gbm.dump_model()


### Parameter tuning

In [ ]:
import math

def TPR_eval_cv( preds,train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    yvalid_scoreordered = labels[np.argsort(preds)]

    N = np.sum(y_test == 0)
    P = np.sum(y_test == 1)
    FP = 0
    TP = 0
    FN = 0
    TN = 0

    for i in range(math.floor(nrows_valid/k) - 1, -1, -1):
        if (yvalid_scoreordered[i] == 1):
            TP = TP + 1
        else:
            FP = FP + 1
            #print("True Positive", TP, "False Positive", FP)
        if (FP / N > 10**-4):
            FP = FP - 1
            break

    return "###TPR(FPR<0.0001)###:", TP/P, True

In [ ]:
# Load training data
nrows_train = 1000000
nrows_valid = 100000
train_valide_n=nrows_train + nrows_valid
xtrain = np.loadtxt('/content/drive/My Drive/DataChallenge2020MSBGD/xtrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_valid)
ytrain = np.loadtxt('/content/drive/My Drive/DataChallenge2020MSBGD/ytrain_challenge.csv', delimiter=',', skiprows = 1, max_rows = nrows_train + nrows_valid)
ytrain = np.array(ytrain).reshape(nrows_train + nrows_valid)

x_train=xtrain[:nrows_train]
y_train=ytrain[:nrows_train]
x_test=xtrain[nrows_train:(nrows_train + nrows_valid)] 
y_test=ytrain[nrows_train:(nrows_train + nrows_valid)] 

In [ ]:
cv_results

In [ ]:
TPR = []
Feature_importance =[]

r = range(1, 14, 1)
global i
i=0
for t in range(1, 14, 1):
    i=i+0.5
    def custom_loss_function(preds, train_data):
        # pred = raw value before logistic transformation <- 0 ->
        y = train_data.get_label()
        pred = preds
        beta = i
        p = 1. / (1. + np.exp(-pred))
        grad = (p * (beta + y - beta * y) - y)
        hess = p*(1.0 - p) * (beta + y - beta * y)
        return grad, hess

    lgb_train = lgb.Dataset(data=x_train, label=y_train)
    lgb_eval = lgb.Dataset(data=x_test, label=y_test, reference=lgb_train)

    params = {
          'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 600,
    'num_trees': 400,
    'learning_rate': 0.05,
    'bagging_fraction': 0.9,
     'scale_pos_weight':0.0969,
    'bagging_freq': 5,
    'verbose': 0
    }

    print(i,'----  Starting training...')
    # train
    gbm = lgb.train(params,lgb_train,valid_sets=lgb_eval,
                    num_boost_round=30,
                    fobj=custom_loss_function,
                    # feval=binary_error,
                    feval=TPR_eval,
                    #learning_rates=lambda iter: 0.05 * (0.99 ** iter),
                    early_stopping_rounds=50)

    print(gbm.best_score)
    print('Starting predicting...')
    # predict
    y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)

     # Check its on another part of the data set
    yvalid = y_pred
    yvalid_scoreordered = y_test[np.argsort(yvalid)]

   
    

In [ ]:
sns.heatmap(confusion_matrix(preds_heatmap, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

### manual tuning with all dataset
I split the data into 9M data train set and the rest for validation set. With the permuation and data augementation the trainning set become 18M. I mainly did the tuning on google Colab.

Finally I used all data including "outliers", I consider these "noise" may help to improve the robustness of the model, and the lightgbm are power enough to handle some noises.


In [ ]:
# data augementation , beta=2, n_leaves=1000, max_depth=10 
def custom_loss_function(preds, train_data):
    y = train_data.get_label()
    beta = 2
    p = np.where(preds >= 0, 1.0 / (1+np.exp(-preds)),
                 np.exp(preds)/(1+np.exp(preds)))
    grad = (p * (beta + y - beta * y) - y)
    hess = p*(1.0 - p) * (beta + y - beta * y)
    return grad, hess


lgb_train = lgb.Dataset(data=x_train,   label=y_train)
lgb_eval = lgb.Dataset(data=x_test, label=y_test, reference=lgb_train)

params = {
    # 'objective' = custom_loss_function,
    'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary',
    'num_leaves': 1000,
    'max_depth': 10, 'learning_rate': 0.1,
    # 'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'scale_pos_weight': 0.0747, 'bagging_freq': 5, 'verbose': 0,   'lambda_l1': 0.1,   'lambda_l2': 0.01,
}
evals_result = {}
print('Starting training...')
gbm = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                num_boost_round=1500,
                feval=TPR_eval,
                fobj=custom_loss_function,
                verbose_eval=10, evals_result=evals_result,
                #learning_rates=lambda iter: 0.10 * (0.996 ** iter),
                #feval=lambda preds, train_data: [accuracy(preds, train_data),TPR_eval(preds, train_data)],
                early_stopping_rounds=50)

print(gbm.best_score)
print('Starting predicting...')
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)def custom_loss_function(preds, train_data):
    y = train_data.get_label()
    beta = 2
    p = np.where(preds >= 0, 1.0 / (1+np.exp(-preds)),
                 np.exp(preds)/(1+np.exp(preds)))
    grad = (p * (beta + y - beta * y) - y)
    hess = p*(1.0 - p) * (beta + y - beta * y)
    return grad, hess

lgb_train = lgb.Dataset(data=x_train,   label=y_train)
lgb_eval = lgb.Dataset(data=x_test, label=y_test, reference=lgb_train)

params = {
    # 'objective' = custom_loss_function,
    'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary',
    'num_leaves': 1000,
    'max_depth': 10, 'learning_rate': 0.1,
    # 'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'scale_pos_weight': 0.0747, 'bagging_freq': 5, 'verbose': 0,   'lambda_l1': 0.1,   'lambda_l2': 0.01,
}
evals_result = {}
print('Starting training...')
gbm = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                num_boost_round=1500,
                feval=TPR_eval,
                fobj=custom_loss_function,
                verbose_eval=10, evals_result=evals_result,
                #learning_rates=lambda iter: 0.10 * (0.996 ** iter),
                #feval=lambda preds, train_data: [accuracy(preds, train_data),TPR_eval(preds, train_data)],
                early_stopping_rounds=50)

print(gbm.best_score)
print('Starting predicting...')
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)

In [ ]:
preds_heatmap=np.where(y_pred>=0.5,1,0)
sns.heatmap(confusion_matrix(preds_heatmap, y_test), annot=True,
            fmt='3.0f', cmap="coolwarm", linewidths=.5)
plt.title('Confusion_matrix', y=1.05, size=15)

# Stacking 
## Ensemble Voting

In [ ]:
# Load test data
xtest = np.loadtxt(
    '/content/drive/My Drive/DataChallenge2020MSBGD/xtest_challenge.csv', delimiter=',', skiprows=1)

In [ ]:
# import the models
clf1 = lgb.Booster(
    model_file='/content/drive/My Drive/DataChallenge2020MSBGD/model_save/final5.txt')
y_pred1 = clf1.predict(xtest, num_iteration=clf1.best_iteration)
# true_positive_rate(y_pred1)
print(y_pred1.shape)

clf2 = lgb.Booster(
    model_file='/content/drive/My Drive/DataChallenge2020MSBGD/model_save/final8.txt')
y_pred2 = clf1.predict(xtest, num_iteration=clf2.best_iteration)

clf3 = lgb.Booster(
    model_file='/content/drive/My Drive/DataChallenge2020MSBGD/model_save/final7.txt')
y_pred3 = clf1.predict(xtest, num_iteration=clf3.best_iteration)

In [ ]:
# manual voting
pred_avg = np.mean(y_pred1, y_pred2, y_pred3)
print(y_pred1[1:5], y_pred2[1:5], y_pred3[1:5])
print(pred_avg[1:5])

np.savetxt('/content/drive/My Drive/DataChallenge2020MSBGD/ytest_challenge_student27A.csv',
           pred_avg, fmt='%1.15f', delimiter=',')

In [ ]:
# explore the different results
# only for test
y1 = np.loadtxt('ytest_challenge_student_WRF5100_alldata.csv',
                delimiter=',', skiprows=1)
y2 = np.loadtxt('ytest_challenge_student7.csv', delimiter=',', skiprows=1)
print(y1[1:5])
print(y2[1:5])

y1 = y1[np.argsort(y1)]
y2 = y2[np.argsort(y1)]

pred1 = np.where(y1 >= 0.5, 1, 0)
pred2 = np.where(y2 >= 0.5, 1, 0)

diff = np.where(pred1 != pred2, y1-y2, 0)

In [ ]:
# diff y1 y2
y3 = (y1+y2)/2
pred3 = np.where(y3 >= 0.5, 1, 0)

diff10_12 = np.where(pred1 > pred2, pred1-pred2, 0)
diff01_12 = np.where(pred1 < pred2, pred1-pred2, 0)
diff10_13 = np.where(pred1 > pred3, pred1-pred3, 0)
diff01_13 = np.where(pred1 < pred3, pred1-pred3, 0)
diff10_23 = np.where(pred2 > pred3, pred2-pred3, 0)
diff01_23 = np.where(pred2 < pred3, pred2-pred3, 0)
print(diff10_12.sum(),
      diff01_12.sum(),
      diff10_13.sum(),
      diff01_13.sum(),
      diff10_23.sum(),
      diff01_23.sum())

print(diff01.sum())

In [ ]:
plt.figure(figsize=(20, 5))
plt.scatter(range(len(diff2)), diff2, alpha=0.1, s=1)

In [ ]:
plt.figure(figsize=(20, 5))
plt.scatter(range(len(diff)), diff, alpha=0.1, s=1)

In [ ]:
plt.figure(figsize=(20,5))
plt.hist(diff2,alpha=0.4, bins=100)

## Stacking 

In [ ]:
stacking_train=np.vstack((y_pred1,y_pred2,y_pred3)).transpose()
stacking_train.shape

In [ ]:
#1st stacking algo 
stacking= LogisticRegression(solver='lbfgs',class_weight={0: 5800, 1: 1})
stacking.fit(stacking_train[:1000000,:], y_train[:1000000])

In [ ]:
# 2nd stacking algo
stacking = RandomForestClassifier(class_weight={0: 5800, 1: 1}, max_features='sqrt',
                                  bootstrap=True,  n_estimators=150,
                                  n_jobs=-1)  # ,  n_estimators=1000)
stacking.fit(stacking_train[:1000000, :], y_train[:1000000])

In [ ]:
#train the stacking model
y_test_stacking1 = clf1.predict(x_test, num_iteration=clf1.best_iteration)

y_test_stacking2 = clf1.predict(x_test, num_iteration=clf2.best_iteration)

y_test_stacking3 = clf1.predict(x_test, num_iteration=clf3.best_iteration)

stacking_test=np.vstack((y_test_stacking1,y_test_stacking2,y_test_stacking3)).transpose()
print(stacking_test.shape)

In [ ]:
# compare the predction of stacking and single classifier
predict1 = clf1.predict(x_test)
predict2 = clf2.predict(x_test)
predict3 = clf3.predict(x_test)

stacking_preds_test = stacking.predict_proba(stacking_test)

In [ ]:
stacking_preds_test=stacking.predict_proba(stacking_test)

# History

## Prepare a file for submission

In [ ]:
# for RF
# Load test data
xtest = np.loadtxt(
    '/content/drive/My Drive/DataChallenge2020MSBGD/xtest_challenge.csv', delimiter=',', skiprows=1)
# Classify the provided test data
ytest = clf.predict_proba(xtest)[:, clf.classes_ == 1][:, 0]
print(ytest.shape)
np.savetxt('/content/drive/My Drive/DataChallenge2020MSBGD/ytest_challenge_student7.csv',
           ytest, fmt='%1.15f', delimiter=',')

In [ ]:
# for GBM

bst = lgb.Booster(
    model_file='/content/drive/My Drive/DataChallenge2020MSBGD//model_save/final5.txt')
# Load test data
xtest = np.loadtxt(
    '/content/drive/My Drive/DataChallenge2020MSBGD/xtest_challenge.csv', delimiter=',', skiprows=1)
# Classify the provided test data
ytest = bst.predict(xtest)  # [:,0]
print(ytest.shape)
print(ytest[1:10])
np.savetxt('/content/drive/My Drive/DataChallenge2020MSBGD/ytest_challenge_studentfinal5.csv',
           ytest, fmt='%1.15f', delimiter=',')

## Submit history



- 2020.2.17 1st summit : 0.8501 
    - ranked 16th/31
    *     Weighted Random Forest with class_weight {5100:1} 


- 2020.2.17 2nd summit : 0.8458 
    - ranked 16th/31  
    -    ( baseline Lightgbm)


- 2020.2.18  summit3 : 0.07 (format error)


- 2020.2.19 summit4 : 0.8730 
    - ranked 9th/34
    *  Lightgbm with L1 L2 regularisation, decay learning rate 
    

- 2020.2.19 summit5 : 0.875682828873 
    - ranked 5th/43
    *  Lightgbm with custom loss function(beta=5), validation=0.9880 
    

- 2020.2.27 summit6: 0.887100497022 
    - ranked 1st/50
    * Lgbm with data augementation , (beta = 2, 1000leaves, max depth=10),validation=0.997287 

